# Fine-tuning VGG16 on flowers17 dataset

In [ ]:
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

from keras_example.preprocessing.imagetoarrayprocessor import ImageToArrayPreprocessor
from keras_example.preprocessing. aspectawareprocessor import AspectAwareProcessor
from keras_example.datasets.simpledatasetloader import SimpleDatasetLoader
from keras_example.nn.conv.fcheadnet import FCHeadNet

from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import RMSprop, SGD

from keras.applications import VGG16
from keras.layers import Input
from keras.models import Model

from imutils import paths

import numpy as np
import os


In [9]:
DATASET_PATH = ''
MODEL_PATH = '' # serialized hdf5 file
MODEL_OUT_PATH = ''
EPOCHS = 25

In [ ]:
aug = ImageDataGenerator(rotation_range=30, width_shift_range=0.1,
    height_shift_range=0.1, shear_range=0.2, zoom_range=0.2,
    horizontal_flip=True, fill_mode='nearest')

In [ ]:
print("[INFO] loading images...")
imagePaths = list(paths.list_images(DATASET_PATH))
classNames = [pt.split(os.path.sep)[-2] for pt in imagePaths]
classNames = [str(x) for x in np.unique(classNames)]

aap = AspectAwareProcessor(224,224)
iap = ImageToArrayPreprocessor()

sdl = SimpleDatasetLoader(preprocessors=[aap, iap])
(data, labels) = sdl.load(imagePaths, verbose = 1)
data = data.astype("float") / 255.0

In [ ]:
(trainImage, testImage,trainLabels,testLabels) = train_test_split(data, labels,
    test_size=0.25, random_state = 42)

trainLabels = LabelBinarizer().fit_transform(trainLabels)
testLabels = LabelBinarizer().transform(testLabels)

In [ ]:
baseModel = VGG16(weights='imagenet', include_top=False, 
    input_tensor=Input(shape=(224,224,3)))

headModel = FCHeadNet.build(baseModel, len(classNames), 256)

model = Model(inputs=baseModel.input, outputs=headModel)

for layer in baseModel.layers:
    layer.trainable = False

print("[INFO] compiling model...")
opt = RMSprop(lr=0.001)

model.compile(loss="categorical-crossentropy", optimizers=opt, 
    metrics=["accuracy"])

print("[INFO] training head...")
model.fit_generator(aug.flow(trainImage, trainLabels, batch_size = 32),
    validation_data=(testImage, testLabels), epochs = EPOCHS,
    steps_per_epoch=len(trainImage) // 32, verbose = 1)

In [ ]:
print("[INFO] evaluating after initialization...")
predictions = model.predict(testImage, batch_size=32)
print(classification_report(testLabels.argmax(axis=1),
    predictions.argmax(axis=1), target_names=classNames))

In [ ]:
for layer in baseModel.layers[15:]:
    layer.trainable = True

print("[INFO] re-compiling model...")
opt = RMSprop(lr=0.001)

model.compile(loss="categorical-crossentropy", optimizers=opt, 
    metrics=["accuracy"])

print("[INFO] fine-tuning model...")
model.fit_generator(aug.flow(trainImage, trainLabels, batch_size = 32),
    validation_data=(testImage, testLabels), epochs = EPOCHS,
    steps_per_epoch=len(trainImage) // 32, verbose = 1)

In [ ]:
print("[INFO] evaluating after initialization...")
predictions = model.predict(testImage, batch_size=32)
print(classification_report(testLabels.argmax(axis=1),
    predictions.argmax(axis=1), target_names=classNames))

model.save(MODEL_OUT_PATH)